# Statistical Significance Testing for Topic-Based Forecasts

## Overview

This notebook performs rigorous statistical testing of forecast results from notebook 07.

**Tests Applied:**
1. **One-Sided Diebold-Mariano** → Tests if ARIMAX significantly outperforms ARIMA
2. **Sign Test** → Tests fold-level consistency (do topics help in majority of folds?)
3. **Wilcoxon Signed-Rank** → Non-parametric test robust to outliers
4. **Bootstrap Diebold-Mariano** → Better small-sample properties
5. **Clark-West Test** → Adjusts for nested models (ARIMAX contains ARIMA)

**Why Multiple Tests?**
- Small sample (n=20 forecasts) requires robust evidence
- Each test has different strengths
- Convergence across tests = stronger conclusion

---

In [1]:
# ============================================================================
# SETUP
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
import warnings
warnings.filterwarnings('ignore')

# Statistical testing
from scipy import stats
from scipy.stats import ttest_rel, wilcoxon, binomtest, norm

# Visualization
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)

print(" Libraries loaded")

 Libraries loaded


---
## Part 1: Load Forecast Results from Notebook 07
---

In [2]:
import pickle

# Load HYBRID results (both d=0 and d=1)
with open('results/forecast_results_HYBRID.pkl', 'rb') as f:
    data = pickle.load(f)

# Extract d=0 (Forecasting)
results_k6_d0 = data['results_k6_d0']
results_k25_d0 = data['results_k25_d0']
summary_k6_d0 = data['summary_k6_d0']
summary_k25_d0 = data['summary_k25_d0']

# Extract d=1 (Causality)
results_k6_d1 = data['results_k6_d1']
results_k25_d1 = data['results_k25_d1']
summary_k6_d1 = data['summary_k6_d1']
summary_k25_d1 = data['summary_k25_d1']

# Metadata
cv_splits = data['cv_splits']

print("="*80)
print("LOADED HYBRID RESULTS")
print("="*80)
print(f"\nd=0 (Forecasting Performance):")
print(f"  K=6:  {len(results_k6_d0)} models")
print(f"  K=25: {len(results_k25_d0)} models")

print(f"\nd=1 (Causal Robustness):")
print(f"  K=6:  {len(results_k6_d1)} models")
print(f"  K=25: {len(results_k25_d1)} models")

print(f"\nCross-validation folds: {len(cv_splits)}")
print("="*80)


LOADED HYBRID RESULTS

d=0 (Forecasting Performance):
  K=6:  62 models
  K=25: 34 models

d=1 (Causal Robustness):
  K=6:  11 models
  K=25: 11 models

Cross-validation folds: 10


---
## Part 2: Statistical Test Functions
---

In [3]:
# ============================================================================
# TEST 1: ONE-SIDED DIEBOLD-MARIANO
# ============================================================================

def diebold_mariano_one_sided(residuals_baseline, residuals_model):
    """
    One-sided Diebold-Mariano test
    
    H0: Model is NOT better than baseline
    H1: Model IS better than baseline (smaller MSE)
    
    Returns:
        dm_stat: Positive if model is better
        p_value: One-sided p-value (small = significant improvement)
    """
    # Loss differential (positive when model beats baseline)
    d = residuals_baseline**2 - residuals_model**2
    
    mean_d = np.mean(d)
    var_d = np.var(d, ddof=1)
    
    if var_d == 0 or len(d) < 2:
        return 0.0, 1.0
    
    # DM statistic
    dm_stat = mean_d / np.sqrt(var_d / len(d))
    
    # One-tailed test (model is better if dm_stat > 0)
    p_value = 1 - stats.t.cdf(dm_stat, df=len(d)-1)
    
    return dm_stat, p_value

# ============================================================================
# TEST 2: SIGN TEST
# ============================================================================

def sign_test_folds(results_baseline, results_model):
    """
    Test if model beats baseline in majority of CV folds
    
    H0: Model wins 50% of folds (random)
    H1: Model wins > 50% of folds
    
    Returns:
        wins: Number of folds where model beats baseline
        total: Total folds
        p_value: Binomial test p-value
    """
    n_folds = len(results_baseline)
    wins = sum(1 for i in range(n_folds) 
               if results_model[i]['rmse'] < results_baseline[i]['rmse'])
    
    # One-sided binomial test
    result = binomtest(wins, n_folds, 0.5, alternative='greater')
    
    return wins, n_folds, result.pvalue

# ============================================================================
# TEST 3: WILCOXON SIGNED-RANK
# ============================================================================

def wilcoxon_test(residuals_baseline, residuals_model):
    """
    Wilcoxon signed-rank test on absolute errors
    
    Non-parametric alternative to paired t-test
    Robust to outliers and non-normal distributions
    
    H0: Median absolute errors are equal
    H1: Model has smaller median absolute error
    """
    abs_err_baseline = np.abs(residuals_baseline)
    abs_err_model = np.abs(residuals_model)
    
    try:
        stat, p_value = wilcoxon(abs_err_baseline, abs_err_model, 
                                 alternative='greater', zero_method='wilcox')
        return stat, p_value
    except:
        return np.nan, np.nan

# ============================================================================
# TEST 4: BOOTSTRAP DIEBOLD-MARIANO
# ============================================================================

def bootstrap_dm_test(residuals_baseline, residuals_model, n_boot=1000, seed=42):
    """
    Bootstrap version of DM test for better small-sample inference
    
    Uses resampling to estimate p-value without relying on 
    asymptotic normal approximation
    """
    np.random.seed(seed)
    
    # Observed statistic
    d_obs = residuals_baseline**2 - residuals_model**2
    mean_obs = d_obs.mean()
    
    # Bootstrap distribution under null (center at zero)
    d_centered = d_obs - mean_obs
    boot_means = []
    
    for _ in range(n_boot):
        idx = np.random.choice(len(d_centered), size=len(d_centered), replace=True)
        boot_means.append(d_centered[idx].mean())
    
    boot_means = np.array(boot_means)
    
    # One-sided p-value
    p_value = (boot_means >= mean_obs).mean()
    
    return mean_obs, p_value

# ============================================================================
# TEST 5: CLARK-WEST TEST
# ============================================================================

def clark_west_test(residuals_baseline, residuals_model, 
                    forecast_baseline, forecast_model):
    """
    Clark-West test for nested models
    
    Adjusts DM test for the fact that ARIMAX has more parameters than ARIMA
    Standard DM test is too conservative for nested models
    
    H0: Baseline is adequate
    H1: Additional parameters (topics) improve fit
    """
    n = len(residuals_baseline)
    
    mse_baseline = residuals_baseline**2
    mse_model = residuals_model**2
    
    # Adjustment for nested models
    adjustment = (forecast_baseline - forecast_model)**2
    
    # CW statistic
    f_bar = (mse_baseline - mse_model + adjustment).mean()
    var_f = (mse_baseline - mse_model + adjustment).var(ddof=1)
    
    if var_f == 0 or n < 2:
        return 0.0, 1.0
    
    cw_stat = f_bar / np.sqrt(var_f / n)
    
    # One-sided test (use normal approximation)
    p_value = 1 - norm.cdf(cw_stat)
    
    return cw_stat, p_value

print("All statistical test functions defined")

All statistical test functions defined


---
## Part 3: Run All Tests on K=6 Models
---

In [4]:
def run_all_tests(results_baseline, results_model, model_name):
    """
    Run all 5 statistical tests on a model
    
    Returns dict with test results
    """
    # Collect data - compute residuals if not present
    residuals_baseline = []
    residuals_model = []
    forecast_baseline = []
    forecast_model = []
    
    for r in results_baseline:
        if 'residuals' in r:
            residuals_baseline.append(r['residuals'])
        else:
            residuals_baseline.append(r['actual'] - r['forecast'])
        forecast_baseline.append(r['forecast'])
    
    for r in results_model:
        if 'residuals' in r:
            residuals_model.append(r['residuals'])
        else:
            residuals_model.append(r['actual'] - r['forecast'])
        forecast_model.append(r['forecast'])
    
    residuals_baseline = np.concatenate(residuals_baseline)
    residuals_model = np.concatenate(residuals_model)
    forecast_baseline = np.concatenate(forecast_baseline)
    forecast_model = np.concatenate(forecast_model)
    
    # Test 1: One-sided DM
    dm_stat, dm_p = diebold_mariano_one_sided(residuals_baseline, residuals_model)
    
    # Test 2: Sign test
    wins, total, sign_p = sign_test_folds(results_baseline, results_model)
    
    # Test 3: Wilcoxon
    wilc_stat, wilc_p = wilcoxon_test(residuals_baseline, residuals_model)
    
    # Test 4: Bootstrap DM
    boot_stat, boot_p = bootstrap_dm_test(residuals_baseline, residuals_model)
    
    # Test 5: Clark-West
    cw_stat, cw_p = clark_west_test(residuals_baseline, residuals_model,
                                     forecast_baseline, forecast_model)
    
    # Calculate performance metrics
    rmse_baseline = np.sqrt(np.mean(residuals_baseline**2))
    rmse_model = np.sqrt(np.mean(residuals_model**2))
    improvement_pct = (rmse_baseline - rmse_model) / rmse_baseline * 100
    
    return {
        'model': model_name,
        'rmse_baseline': rmse_baseline,
        'rmse_model': rmse_model,
        'improvement_%': improvement_pct,
        'folds_better': f"{wins}/{total}",
        'win_rate': wins/total,
        'dm_stat': dm_stat,
        'dm_p': dm_p,
        'sign_p': sign_p,
        'wilcoxon_p': wilc_p,
        'bootstrap_p': boot_p,
        'clarkwest_p': cw_p,
        'dm_sig': dm_p < 0.05,
        'sign_sig': sign_p < 0.05,
        'wilcoxon_sig': wilc_p < 0.05 if not np.isnan(wilc_p) else False,
        'bootstrap_sig': boot_p < 0.05,
        'clarkwest_sig': cw_p < 0.05,
        'n_tests_sig': sum([
            dm_p < 0.05,
            sign_p < 0.05,
            wilc_p < 0.05 if not np.isnan(wilc_p) else False,
            boot_p < 0.05,
            cw_p < 0.05
        ])
    }

print("="*80)
print("RUNNING STATISTICAL TESTS: K=6 MODELS")
print("="*80)
print(f"\nTesting {len(results_k6_d0)-1} models against baseline...\n")

test_results_k6_d0 = []
baseline_k6_d0 = results_k6_d0['Baseline']

for i, (model_name, model_results) in enumerate(results_k6_d0.items(), 1):
    if model_name == 'Baseline':
        continue
    
    print(f"[{i-1:2d}/{len(results_k6_d0)-1}] Testing {model_name:25s}", end=" ")
    
    test_result = run_all_tests(baseline_k6_d0, model_results, model_name)
    test_results_k6_d0.append(test_result)
    
    sig_count = test_result['n_tests_sig']
    print(f"→ {sig_count}/5 tests significant")

df_tests_k6_d0 = pd.DataFrame(test_results_k6_d0)
df_tests_k6_d0 = df_tests_k6_d0.sort_values('n_tests_sig', ascending=False)

print(f"\n K=6 testing complete")
print(f"  Models with ANY significant test: {(df_tests_k6_d0['n_tests_sig'] > 0).sum()}/{len(df_tests_k6_d0)}")
print(f"  Models with MAJORITY significant (3+): {(df_tests_k6_d0['n_tests_sig'] >= 3).sum()}/{len(df_tests_k6_d0)}")

RUNNING STATISTICAL TESTS: K=6 MODELS

Testing 61 models against baseline...

[ 1/61] Testing K6_T2                     → 0/5 tests significant
[ 2/61] Testing K6_T3                     → 4/5 tests significant
[ 3/61] Testing K6_T4                     → 0/5 tests significant
[ 4/61] Testing K6_T5                     → 1/5 tests significant
[ 5/61] Testing K6_T6                     → 4/5 tests significant
[ 6/61] Testing K6_T1_T3                  → 0/5 tests significant
[ 7/61] Testing K6_T1_T4                  → 0/5 tests significant
[ 8/61] Testing K6_T1_T5                  → 0/5 tests significant
[ 9/61] Testing K6_T1_T6                  → 4/5 tests significant
[10/61] Testing K6_T2_T3                  → 4/5 tests significant
[11/61] Testing K6_T2_T4                  → 0/5 tests significant
[12/61] Testing K6_T2_T5                  → 2/5 tests significant
[13/61] Testing K6_T2_T6                  → 4/5 tests significant
[14/61] Testing K6_T3_T4                  → 4/5 tests significan

---
## Part 4: Run All Tests on K=25 Models
---

In [5]:
print("="*80)
print("RUNNING STATISTICAL TESTS: K=25 MODELS")
print("="*80)
print(f"\nTesting {len(results_k25_d0)-1} models against baseline...\n")

test_results_k25_d0 = []
baseline_k25_d0 = results_k25_d0['Baseline']

for i, (model_name, model_results) in enumerate(results_k25_d0.items(), 1):
    if model_name == 'Baseline':
        continue
    
    print(f"[{i-1:2d}/{len(results_k25_d0)-1}] Testing {model_name:25s}", end=" ")
    
    test_result = run_all_tests(baseline_k25_d0, model_results, model_name)
    test_results_k25_d0.append(test_result)
    
    sig_count = test_result['n_tests_sig']
    print(f"→ {sig_count}/5 tests significant")

df_tests_k25_d0 = pd.DataFrame(test_results_k25_d0)
df_tests_k25_d0 = df_tests_k25_d0.sort_values('n_tests_sig', ascending=False)

print(f"\n K=25 testing complete")
print(f"  Models with ANY significant test: {(df_tests_k25_d0['n_tests_sig'] > 0).sum()}/{len(df_tests_k25_d0)}")
print(f"  Models with MAJORITY significant (3+): {(df_tests_k25_d0['n_tests_sig'] >= 3).sum()}/{len(df_tests_k25_d0)}")

RUNNING STATISTICAL TESTS: K=25 MODELS

Testing 33 models against baseline...

[ 1/33] Testing K25_T1                    → 0/5 tests significant
[ 2/33] Testing K25_T4                    → 0/5 tests significant
[ 3/33] Testing K25_T7                    → 0/5 tests significant
[ 4/33] Testing K25_T8                    → 0/5 tests significant
[ 5/33] Testing K25_T9                    → 0/5 tests significant
[ 6/33] Testing K25_T10                   → 4/5 tests significant
[ 7/33] Testing K25_T15                   → 0/5 tests significant
[ 8/33] Testing K25_T20                   → 0/5 tests significant
[ 9/33] Testing K25_T24                   → 4/5 tests significant
[10/33] Testing K25_T1_T10                → 0/5 tests significant
[11/33] Testing K25_T1_T24                → 0/5 tests significant
[12/33] Testing K25_T1_T17                → 0/5 tests significant
[13/33] Testing K25_T1_T20                → 0/5 tests significant
[14/33] Testing K25_T10_T24               → 4/5 tests significa

---
## Part 5: Detailed Results Tables
---

In [6]:
# K=6 Results
print("="*80)
print("STATISTICAL TEST RESULTS: K=6 (Top 10 Models)")
print("="*80)

display_cols = ['model', 'improvement_%', 'folds_better', 'n_tests_sig',
                'dm_p', 'sign_p', 'wilcoxon_p', 'bootstrap_p', 'clarkwest_p']
print("\n" + df_tests_k6_d0[display_cols].head(10).to_string(index=False))

print("\n" + "="*80)
print("Significance Summary (p < 0.05):")
print("="*80)
sig_summary_k6 = df_tests_k6_d0[['model', 'dm_sig', 'sign_sig', 'wilcoxon_sig', 
                               'bootstrap_sig', 'clarkwest_sig', 'n_tests_sig']].head(10)
print("\n" + sig_summary_k6.to_string(index=False))

STATISTICAL TEST RESULTS: K=6 (Top 10 Models)

               model  improvement_% folds_better  n_tests_sig     dm_p   sign_p  wilcoxon_p  bootstrap_p  clarkwest_p
K6_T1_T2_T3_T4_T5_T6       8.791416         9/10            5 0.003960 0.010742    0.000931        0.008     0.000516
      K6_T2_T3_T5_T6       9.931547         9/10            5 0.002460 0.010742    0.000422        0.005     0.000298
         K6_T1_T3_T6       8.765706         9/10            5 0.004683 0.010742    0.000422        0.008     0.001221
         K6_T3_T4_T6       8.628277         9/10            5 0.005731 0.010742    0.000649        0.014     0.001205
         K6_T3_T5_T6       9.859040         9/10            5 0.002948 0.010742    0.000553        0.006     0.000605
      K6_T1_T2_T3_T6       9.391227         9/10            5 0.002801 0.010742    0.000240        0.006     0.000332
      K6_T1_T3_T4_T6       8.285513         9/10            5 0.006770 0.010742    0.001454        0.013     0.001566
         

In [7]:
# K=25 Results
print("="*80)
print("STATISTICAL TEST RESULTS: K=25 (Top 10 Models)")
print("="*80)

print("\n" + df_tests_k25_d0[display_cols].head(10).to_string(index=False))

print("\n" + "="*80)
print("Significance Summary (p < 0.05):")
print("="*80)
sig_summary_k25 = df_tests_k25_d0[['model', 'dm_sig', 'sign_sig', 'wilcoxon_sig',
                                 'bootstrap_sig', 'clarkwest_sig', 'n_tests_sig']].head(10)
print("\n" + sig_summary_k25.to_string(index=False))

STATISTICAL TEST RESULTS: K=25 (Top 10 Models)

          model  improvement_% folds_better  n_tests_sig     dm_p   sign_p  wilcoxon_p  bootstrap_p  clarkwest_p
    K25_T10_T20       4.679384         9/10            5 0.001045 0.010742    0.000226        0.002     0.000162
        K25_T10       1.019130         8/10            4 0.016551 0.054688    0.005099        0.015     0.008838
K25_T10_T17_T24       2.677354         8/10            4 0.001396 0.054688    0.003792        0.001     0.000193
    K25_T10_T17       1.199070         8/10            4 0.010514 0.054688    0.021097        0.006     0.003880
        K25_T24       0.537292         8/10            4 0.047955 0.054688    0.011603        0.028     0.036504
    K25_T10_T24       2.309901         8/10            4 0.004021 0.054688    0.001196        0.002     0.000995
    K25_T17_T24       0.497075         8/10            3 0.039379 0.054688    0.113254        0.022     0.020340
 K25_T1_T10_T24       2.636790         7/10     

---
## Part 8: Interpretation and Recommendations
---

In [8]:
def generate_interpretation(df_tests, model_family):
    """
    Generate interpretation and recommended narrative
    """
    print("="*80)
    print(f"INTERPRETATION: {model_family}")
    print("="*80)
    
    # Best model by evidence
    best_model = df_tests.iloc[0]
    
    print(f"\n[1] BEST MODEL: {best_model['model']}")
    print(f"    RMSE Improvement: {best_model['improvement_%']:.1f}%")
    print(f"    Folds better: {best_model['folds_better']}")
    print(f"    Tests significant: {best_model['n_tests_sig']}/5")
    print(f"\n    P-values:")
    print(f"      Diebold-Mariano:  {best_model['dm_p']:.4f} {'✓' if best_model['dm_sig'] else '✗'}")
    print(f"      Sign Test:        {best_model['sign_p']:.4f} {'✓' if best_model['sign_sig'] else '✗'}")
    print(f"      Wilcoxon:         {best_model['wilcoxon_p']:.4f} {'✓' if best_model['wilcoxon_sig'] else '✗'}")
    print(f"      Bootstrap DM:     {best_model['bootstrap_p']:.4f} {'✓' if best_model['bootstrap_sig'] else '✗'}")
    print(f"      Clark-West:       {best_model['clarkwest_p']:.4f} {'✓' if best_model['clarkwest_sig'] else '✗'}")
    
    # Overall statistics
    n_models = len(df_tests)
    n_any_sig = (df_tests['n_tests_sig'] > 0).sum()
    n_majority_sig = (df_tests['n_tests_sig'] >= 3).sum()
    n_all_sig = (df_tests['n_tests_sig'] == 5).sum()
    
    print(f"\n[2] OVERALL ASSESSMENT")
    print(f"    Models tested: {n_models}")
    print(f"    Models with ANY significant test: {n_any_sig}/{n_models} ({n_any_sig/n_models*100:.0f}%)")
    print(f"    Models with MAJORITY significant (3+/5): {n_majority_sig}/{n_models} ({n_majority_sig/n_models*100:.0f}%)")
    print(f"    Models with ALL tests significant (5/5): {n_all_sig}/{n_models}")
    
    avg_improvement = df_tests.head(5)['improvement_%'].mean()
    print(f"\n    Average improvement (top 5): {avg_improvement:.1f}%")
    
    # Test agreement
    print(f"\n[3] TEST AGREEMENT (Top 10 models)")
    top_10 = df_tests.head(10)
    print(f"    DM test:         {top_10['dm_sig'].sum()}/10 significant")
    print(f"    Sign test:       {top_10['sign_sig'].sum()}/10 significant")
    print(f"    Wilcoxon test:   {top_10['wilcoxon_sig'].sum()}/10 significant")
    print(f"    Bootstrap DM:    {top_10['bootstrap_sig'].sum()}/10 significant")
    print(f"    Clark-West:      {top_10['clarkwest_sig'].sum()}/10 significant")



---
# PART 5: Statistical Tests for d=1 (Causal Robustness)

Test if improvements persist when using ARIMAX(2,1,0) with differencing.

---

## Part 5.1: K=6 Tests (d=1)

In [9]:
print("="*80)
print("K=6 STATISTICAL TESTS (d=1)")
print("="*80)
print(f"\nTesting {len(results_k6_d1)-1} models against baseline...\n")

test_results_k6_d1 = []
baseline_k6_d1 = results_k6_d1['Baseline']

for i, (model_name, model_results) in enumerate(results_k6_d1.items(), 1):
    if model_name == 'Baseline':
        continue
    
    print(f"[{i-1:2d}/{len(results_k6_d1)-1}] Testing {model_name:25s}", end=" ")
    
    test_result = run_all_tests(baseline_k6_d1, model_results, model_name)
    test_results_k6_d1.append(test_result)
    
    sig_count = test_result['n_tests_sig']
    print(f"→ {sig_count}/5 tests significant")

df_tests_k6_d1 = pd.DataFrame(test_results_k6_d1)
df_tests_k6_d1 = df_tests_k6_d1.sort_values('n_tests_sig', ascending=False)

print(f"\n K=6 d=1 testing complete")
print(f"  Models with ANY significant test: {(df_tests_k6_d1['n_tests_sig'] > 0).sum()}/{len(df_tests_k6_d1)}")
print(f"  Models with MAJORITY significant (3+): {(df_tests_k6_d1['n_tests_sig'] >= 3).sum()}/{len(df_tests_k6_d1)}")


K=6 STATISTICAL TESTS (d=1)

Testing 10 models against baseline...

[ 1/10] Testing K6_T3                     → 0/5 tests significant
[ 2/10] Testing K6_T2_T3                  → 0/5 tests significant
[ 3/10] Testing K6_T2_T5                  → 0/5 tests significant
[ 4/10] Testing K6_T3_T5                  → 0/5 tests significant
[ 5/10] Testing K6_T4_T5                  → 0/5 tests significant
[ 6/10] Testing K6_T1_T2_T3               → 0/5 tests significant
[ 7/10] Testing K6_T2_T3_T5               → 0/5 tests significant
[ 8/10] Testing K6_T2_T4_T5               → 0/5 tests significant
[ 9/10] Testing K6_T1_T2_T3_T5            → 0/5 tests significant
[10/10] Testing K6_T2_T3_T4_T5            → 0/5 tests significant

 K=6 d=1 testing complete
  Models with ANY significant test: 0/10
  Models with MAJORITY significant (3+): 0/10


## Part 5.2: K=25 Tests (d=1)

In [10]:
print("="*80)
print("K=25 STATISTICAL TESTS (d=1)")
print("="*80)
print(f"\nTesting {len(results_k25_d1)-1} models against baseline...\n")

test_results_k25_d1 = []
baseline_k25_d1 = results_k25_d1['Baseline']

for i, (model_name, model_results) in enumerate(results_k25_d1.items(), 1):
    if model_name == 'Baseline':
        continue
    
    print(f"[{i-1:2d}/{len(results_k25_d1)-1}] Testing {model_name:25s}", end=" ")
    
    test_result = run_all_tests(baseline_k25_d1, model_results, model_name)
    test_results_k25_d1.append(test_result)
    
    sig_count = test_result['n_tests_sig']
    print(f"→ {sig_count}/5 tests significant")

df_tests_k25_d1 = pd.DataFrame(test_results_k25_d1)
df_tests_k25_d1 = df_tests_k25_d1.sort_values('n_tests_sig', ascending=False)

print(f"\n K=25 d=1 testing complete")
print(f"  Models with ANY significant test: {(df_tests_k25_d1['n_tests_sig'] > 0).sum()}/{len(df_tests_k25_d1)}")
print(f"  Models with MAJORITY significant (3+): {(df_tests_k25_d1['n_tests_sig'] >= 3).sum()}/{len(df_tests_k25_d1)}")


K=25 STATISTICAL TESTS (d=1)

Testing 10 models against baseline...

[ 1/10] Testing K25_T9                    → 0/5 tests significant
[ 2/10] Testing K25_T10                   → 0/5 tests significant
[ 3/10] Testing K25_T15                   → 0/5 tests significant
[ 4/10] Testing K25_T20                   → 0/5 tests significant
[ 5/10] Testing K25_T24                   → 0/5 tests significant
[ 6/10] Testing K25_T10_T24               → 0/5 tests significant
[ 7/10] Testing K25_T10_T20               → 0/5 tests significant
[ 8/10] Testing K25_T20_T24               → 0/5 tests significant
[ 9/10] Testing K25_T13_T24               → 0/5 tests significant
[10/10] Testing K25_T10_T17_T24           → 0/5 tests significant

 K=25 d=1 testing complete
  Models with ANY significant test: 0/10
  Models with MAJORITY significant (3+): 0/10


---
# PART 6: Comparison - d=0 vs d=1

## Critical Question: Which models are significant in BOTH specifications?

---

In [11]:
print("="*80)
print("COMPARISON: SIGNIFICANCE IN d=0 vs d=1")
print("="*80)

# For K=6, compare which models are significant in both

print("\n K=6 COMPARISON")
print("-" * 80)

# Get significant models from each (majority of tests significant: >= 3 out of 5)
sig_d0 = set(df_tests_k6_d0[df_tests_k6_d0['n_tests_sig'] >= 3]['model'].values)
sig_d1 = set(df_tests_k6_d1[df_tests_k6_d1['n_tests_sig'] >= 3]['model'].values)

# Find overlap
sig_both = sig_d0.intersection(sig_d1)
sig_d0_only = sig_d0 - sig_d1
sig_d1_only = sig_d1 - sig_d0

print(f"Significant in d=0 (Forecasting):     {len(sig_d0)} models")
print(f"Significant in d=1 (Causality):       {len(sig_d1)} models")
print(f"Significant in BOTH:                  {len(sig_both)} models")
print(f"Significant in d=0 only:              {len(sig_d0_only)} models")
print(f"Significant in d=1 only:              {len(sig_d1_only)} models")

print(f"\nModels significant in BOTH (strongest evidence):")
for model in list(sig_both)[:10]:  # Top 10
    imp_d0 = df_tests_k6_d0[df_tests_k6_d0['model'] == model]['improvement_%'].values[0]
    imp_d1 = df_tests_k6_d1[df_tests_k6_d1['model'] == model]['improvement_%'].values[0]
    print(f"  {model:25s} d=0: {imp_d0:6.2f}%  d=1: {imp_d1:6.2f}%")


COMPARISON: SIGNIFICANCE IN d=0 vs d=1

 K=6 COMPARISON
--------------------------------------------------------------------------------
Significant in d=0 (Forecasting):     43 models
Significant in d=1 (Causality):       0 models
Significant in BOTH:                  0 models
Significant in d=0 only:              43 models
Significant in d=1 only:              0 models

Models significant in BOTH (strongest evidence):


---
## Part 9: Export Results
---

In [13]:
# Save test results
df_tests_k6_d0.to_csv('results/statistical_tests_k6.csv', index=False)
df_tests_k25_d0.to_csv('results/statistical_tests_k25.csv', index=False)

print("="*80)
print("RESULTS SAVED")
print("="*80)
print(f"✓ results/statistical_tests_k6.csv")
print(f"✓ results/statistical_tests_k25.csv")
print("="*80)

RESULTS SAVED
✓ results/statistical_tests_k6.csv
✓ results/statistical_tests_k25.csv
